In [1]:
import timeit
from datetime import datetime
import requests
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import urllib
import json

In [2]:
if "auth_cookie" not in locals():
    auth_cookie = None

autorun_execute_count = 0
columns = ['EPOCH', 'NORAD_CAT_ID', 'MEAN_MOTION', 'ECCENTRICITY', 'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS']

In [3]:
try:
    logs = pd.read_pickle("./logs.pkl.gz","gzip")
except:
    # time, type (trigger, login, skip, fetch_init, fetch_success, process_complete, error), url, epoch, out file
    logs = pd.DataFrame(columns = ["created_on","type","url","epoch","error","output"])
    logs = logs.astype({'created_on': 'datetime64[ns]'})
    
def save_logs():
    logs.to_pickle("./logs.pkl.gz","gzip")
    
def log(log_type,url=None,epoch=None,error=None,output=None):
    global logs
    new_log = {'created_on':np.datetime64('now'), 'type':log_type, 'url':url, 'epoch':epoch, 'error':error, 'output':output}
    logs = logs.append(new_log,ignore_index=True)
    save_logs()

In [25]:
def get_current_file_count():
    return max([int(f[:-7]) for f in listdir('./output/') if f.endswith(".pkl.gz")])

def get_last_epoch():
    df = pd.read_pickle("./output/%05d.pkl.gz" % get_current_file_count(),"gzip")
    return df.EPOCH[-1:].astype("datetime64[ns]").values[0]

def get_space_track_api_count(unit="m"):
    return len(logs[(logs.created_on > (np.datetime64('now') - np.timedelta64(1,unit))) & (logs.type=="fetch_init")])

def space_track_login():
    with open("space-track-credentials.json") as json_file:
        credentials = json.load(json_file)
    print("Logging in space-track.org with identity:",credentials['identity'])
    url = "https://www.space-track.org/ajaxauth/login"
    x = requests.post(url, data = credentials)
    if x.status_code == 200:
        print("Logged in")
        log("login", output="success")
        return x.cookies
    else:
        log("login", error=x.status_code)
        print("Failed with code:",x.status_code)
        return None
    
def generate_url():
    lower = get_last_epoch()
    l = np.datetime_as_string(lower, unit='us')
    if lower.astype('datetime64[Y]').item().year > 2014:
        upper = lower + np.timedelta64(60,'D') # after 2014
    else:
        upper = lower + np.timedelta64(365,'D') # before 2014
    u = np.datetime_as_string(upper, unit='us')
    return (l, u, f'https://www.space-track.org/basicspacedata/query/class/gp_history/ECCENTRICITY/<0.25/MEAN_MOTION//>11.25/EPOCH/>{l},<{u}/OBJECT_TYPE/DEBRIS/orderby/EPOCH asc/limit/100000/format/csv/emptyresult/show')

Before you proceed:
* Create the `space-track-credentials.json` file from template with your own credentials
* Manually test the fetch cell to make sure things are working first
* Set an appropriate intervalTime
* This will probably mess up if your task time is longer than your interval time
* If you don't know what you are doing, ask Tim.

In [5]:
raise Exception('This exception is to stop the cells below from running when you do "Run All Cells"')

Exception: This exception is to stop the cells below from running when you do "Run All Cells"

In [ ]:
%%html
<script>
    var intervalTime = 300000; // 5 minutes in ms
    if (typeof autorun_toggle === 'undefined') {
        var autorun_toggle = null;
    }
    function toggle_autorun() {
        var btn = document.getElementById("autorun_button");
        setTimeout(function(){ // add a delay so the selected cell is correctly set here
            if (autorun_toggle == null) {
                var start_index = IPython.notebook.get_selected_index()+1;
                var end_index = IPython.notebook.get_cells().length;
                console.log("start autorun, start: " + start_index + " end: " + end_index);
                IPython.notebook.execute_cell_range(start_index, end_index);
                //$('div.input').hide(200);
                autorun_toggle = setInterval(function(){
                    console.log("Run cells below");
                    //IPython.notebook.execute_cells_below();
                    IPython.notebook.execute_cell_range(start_index, end_index);
                }, intervalTime);
            } else {
                window.clearInterval(autorun_toggle);
                console.log("clear autorun");
                autorun_toggle = null;
                //$('div.input').show(200);
            }
        }, 100);
    }
</script>
<button id="autorun_button" onclick="toggle_autorun()">Toggle autorun cells below</button>

In [ ]:
start_time = timeit.default_timer()
autorun_execute_count+=1
print(f'Last run local time: {datetime.now()}')
print(f'Total times ran: {autorun_execute_count}')

In [28]:
# Check that we did not exceed Space-Track API limits
# Limit API queries to less than 30 requests per minute / 300 requests per hour

ops_count = 0

while get_space_track_api_count("m") < 25 and get_space_track_api_count("h") < 275:
    ops_count += 1
    # clear expired cookies
    print("------")
    print(f"Local time: {datetime.now()}")
    if type(auth_cookie) == requests.cookies.RequestsCookieJar:
        auth_cookie.clear_expired_cookies()
    # log in if needed
    if auth_cookie == None or len(auth_cookie) == 0:
        auth_cookie = space_track_login()
    # stop if can't log in
    if auth_cookie == None:
        log("skip", error = f'Cannot log in')
        break
    # prep url
    start, _, url = generate_url()
    print(f"Fetch at epoch >: {start}")
    log("fetch_init", url=url, epoch=start)
    r = requests.get(url, allow_redirects=True, cookies=auth_cookie)
    fname = f"./tmp/{start}.csv".replace(":","_")
    open(fname, 'wb').write(r.content)
    print(f"Saved CSV: {fname}")
    log("fetch_success", url=url, epoch=start, output=fname)
    oname = ("./output/%05d.pkl.gz" % (get_current_file_count()+1)).replace(":","_")
    pd.read_csv(fname)[columns].to_pickle(oname,"gzip")
    log("process_complete", url=url, epoch=start, output=oname)
    print(f"Processed to: {oname}")
    
if ops_count == 0:
    log("skip", error = f'Request limit exceeded: 1m({get_space_track_api_count("m")}) 1h({get_space_track_api_count("h")})')
else:
    log("stop", output=ops_count, error = f'Request limit exceeded: 1m({get_space_track_api_count("m")}) 1h({get_space_track_api_count("h")})')

------
Local time: 2021-01-19 09:53:41.239982
Fetch at epoch >: 2017-12-02T08:58:41.880576
Saved CSV: ./tmp/2017-12-02T08_58_41.880576.csv
Processed to: ./output/00622.pkl.gz
------
Local time: 2021-01-19 09:54:10.937000
Fetch at epoch >: 2017-12-08T22:38:18.953952
Saved CSV: ./tmp/2017-12-08T22_38_18.953952.csv
Processed to: ./output/00623.pkl.gz
------
Local time: 2021-01-19 09:54:48.260790
Fetch at epoch >: 2017-12-15T12:00:54.894240
Saved CSV: ./tmp/2017-12-15T12_00_54.894240.csv
Processed to: ./output/00624.pkl.gz
------
Local time: 2021-01-19 09:55:20.289414
Fetch at epoch >: 2017-12-23T05:57:18.738720
Saved CSV: ./tmp/2017-12-23T05_57_18.738720.csv
Processed to: ./output/00625.pkl.gz
------
Local time: 2021-01-19 09:55:53.399616
Fetch at epoch >: 2017-12-30T23:10:47.641152
Saved CSV: ./tmp/2017-12-30T23_10_47.641152.csv
Processed to: ./output/00626.pkl.gz
------
Local time: 2021-01-19 09:56:27.787273
Fetch at epoch >: 2018-01-06T20:05:45.609216
Saved CSV: ./tmp/2018-01-06T20_05_

/Users/tim/opt/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processed to: ./output/00641.pkl.gz
------
Local time: 2021-01-19 10:05:28.232805
Fetch at epoch >: 2018-05-03T04:50:40.269408
Saved CSV: ./tmp/2018-05-03T04_50_40.269408.csv
Processed to: ./output/00642.pkl.gz
------
Local time: 2021-01-19 10:06:08.076251
Fetch at epoch >: 2018-05-10T19:15:30.733056
Saved CSV: ./tmp/2018-05-10T19_15_30.733056.csv
Processed to: ./output/00643.pkl.gz
------
Local time: 2021-01-19 10:06:49.256798
Fetch at epoch >: 2018-05-18T03:57:34.918272
Saved CSV: ./tmp/2018-05-18T03_57_34.918272.csv
Processed to: ./output/00644.pkl.gz
------
Local time: 2021-01-19 10:07:25.815168
Fetch at epoch >: 2018-05-25T20:10:53.691744
Saved CSV: ./tmp/2018-05-25T20_10_53.691744.csv
Processed to: ./output/00645.pkl.gz
------
Local time: 2021-01-19 10:08:12.297394
Fetch at epoch >: 2018-06-01T04:09:43.991712
Saved CSV: ./tmp/2018-06-01T04_09_43.991712.csv
Processed to: ./output/00646.pkl.gz
------
Local time: 2021-01-19 10:08:46.417664
Fetch at epoch >: 2018-06-08T16:56:26.88288

/Users/tim/opt/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processed to: ./output/00650.pkl.gz
------
Local time: 2021-01-19 10:11:00.025105
Fetch at epoch >: 2018-07-09T01:43:03.997056
Saved CSV: ./tmp/2018-07-09T01_43_03.997056.csv
Processed to: ./output/00651.pkl.gz
------
Local time: 2021-01-19 10:11:34.275896
Fetch at epoch >: 2018-07-18T10:51:34.034400
Saved CSV: ./tmp/2018-07-18T10_51_34.034400.csv
Processed to: ./output/00652.pkl.gz
------
Local time: 2021-01-19 10:12:09.703452
Fetch at epoch >: 2018-07-26T18:05:07.177920
Saved CSV: ./tmp/2018-07-26T18_05_07.177920.csv
Processed to: ./output/00653.pkl.gz
------
Local time: 2021-01-19 10:12:41.869473
Fetch at epoch >: 2018-08-05T14:55:11.727264
Saved CSV: ./tmp/2018-08-05T14_55_11.727264.csv
Processed to: ./output/00654.pkl.gz
------
Local time: 2021-01-19 10:13:25.268352
Fetch at epoch >: 2018-08-15T11:56:03.453216
Saved CSV: ./tmp/2018-08-15T11_56_03.453216.csv
Processed to: ./output/00655.pkl.gz
------
Local time: 2021-01-19 10:14:03.231077
Fetch at epoch >: 2018-08-24T22:00:29.34000

...

Processed to: ./output/00704.pkl.gz
------
Local time: 2021-01-19 10:45:07.462432
Fetch at epoch >: 2019-11-27T00:28:48.065664
Saved CSV: ./tmp/2019-11-27T00_28_48.065664.csv


/Users/tim/opt/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processed to: ./output/00705.pkl.gz
------
Local time: 2021-01-19 10:45:44.758553
Fetch at epoch >: 2019-12-06T04:04:15.814272
Saved CSV: ./tmp/2019-12-06T04_04_15.814272.csv
Processed to: ./output/00706.pkl.gz
------
Local time: 2021-01-19 10:46:28.618964
Fetch at epoch >: 2019-12-15T08:06:03.886272
Saved CSV: ./tmp/2019-12-15T08_06_03.886272.csv
Processed to: ./output/00707.pkl.gz
------
Local time: 2021-01-19 10:47:09.325750
Fetch at epoch >: 2019-12-24T05:38:41.234208
Saved CSV: ./tmp/2019-12-24T05_38_41.234208.csv
Processed to: ./output/00708.pkl.gz
------
Local time: 2021-01-19 10:47:46.810340
Fetch at epoch >: 2020-01-01T21:16:10.373952
Saved CSV: ./tmp/2020-01-01T21_16_10.373952.csv
Processed to: ./output/00709.pkl.gz
------
Local time: 2021-01-19 10:48:24.681794
Fetch at epoch >: 2020-01-10T04:28:31.760544
Saved CSV: ./tmp/2020-01-10T04_28_31.760544.csv
Processed to: ./output/00710.pkl.gz
------
Local time: 2021-01-19 10:49:06.622219
Fetch at epoch >: 2020-01-18T15:59:26.43302

Processed to: ./output/00752.pkl.gz
------
Local time: 2021-01-19 11:16:29.614953
Fetch at epoch >: 2020-10-25T12:50:31.179840
Saved CSV: ./tmp/2020-10-25T12_50_31.179840.csv
Processed to: ./output/00753.pkl.gz
------
Local time: 2021-01-19 11:17:05.919846
Fetch at epoch >: 2020-10-31T17:20:35.195712
Saved CSV: ./tmp/2020-10-31T17_20_35.195712.csv
Processed to: ./output/00754.pkl.gz
------
Local time: 2021-01-19 11:17:33.207695
Fetch at epoch >: 2020-11-06T08:54:47.905920
Saved CSV: ./tmp/2020-11-06T08_54_47.905920.csv
Processed to: ./output/00755.pkl.gz
------
Local time: 2021-01-19 11:18:16.985336
Fetch at epoch >: 2020-11-16T03:25:25.851648
Saved CSV: ./tmp/2020-11-16T03_25_25.851648.csv
Processed to: ./output/00756.pkl.gz
------
Local time: 2021-01-19 11:18:54.564859
Fetch at epoch >: 2020-11-22T13:15:26.306784
Saved CSV: ./tmp/2020-11-22T13_15_26.306784.csv
Processed to: ./output/00757.pkl.gz
------
Local time: 2021-01-19 11:19:31.428134
Fetch at epoch >: 2020-11-28T04:21:31.47379

KeyboardInterrupt: 

In [ ]:
elapsed = timeit.default_timer() - start_time
print(f"Time used to execute cells: {elapsed} seconds")

In [ ]:
logs

In [24]:
get_last_epoch().astype('datetime64[Y]').item().year

int